### Setup the environment
https://joelmccune.com/anaconda-and-arcgis-pro/
create a file named `enviroment.yml` containing the following text

```
# https://joelmccune.com/anaconda-and-arcgis-pro/
name: arcgis

channels:
  - esri 
  - conda-forge
  - defaults
  
dependencies:
  - arcgis
  - arcpy
  - deep-learning-essentials
  - cookiecutter
  - nodejs
  - python=3.7
```

open command prompt as administrator
navigate to directory where environment.yml is saved

```
> conda env create -f [C:\path\to\your\]environment.yml
> conda activate arcgis
```

updating the environment, edit the enviromnent.yml file then navigate to the directory and enter the following command in the terminal as administrator. 

```
conda env update --prefix arcgis --file environment.yml  --prune
```

### Preprpcessing 

1. Union > Dissolve Drago 500yr floodplains
2. Project Trueheart to Drago, 
3. Union Trueheart and Drago
4. Dissolve Imperv > calculate field Imperv = 1, Feature to Raster

In [34]:
# Reset environment settings to default settings.
arcpy.ResetEnvironments()

# setup work space
import arcpy
import os
import glob
import tempfile
# Set environment settings
arcpy.env.workspace = "C:/Workspace/VTWetlands/VTWetlands/"
wd = arcpy.env.workspace + "/"
print(os.listdir(wd))
print(arcpy.env.workspace)
outfolder = arcpy.env.workspace + "/outputs/"
print(outfolder)

# random number generator and seed 
arcpy.env.randomGenerator = "99 ACM599"

# set the number of cores to distribute processing to
arcpy.env.parallelProcessingFactor = "50%"

# output coordinate system
template_SRS = r"D:\GeoData\LandLandcov_BaseLC2016\LandLandcov_BaseLC2016.tif"
spatial_ref = arcpy.Describe(template_SRS).spatialReference
arcpy.env.outputCoordinateSystem = spatial_ref 

import pandas as pd 
import numpy as np
from shapely.geometry import Point
from arcgis.geometry import Geometry
import math


# read in list of selected sites and shuffle values
df = pd.read_csv(wd+"sites/selected_sites_2021-03-04.csv")
roster = pd.read_csv(wd+"sites/roster.csv")
netid = roster.NETID
nstudents = len(netid)
print(netid)
print(df)
nreps = 7


['.backups', '.ipynb_checkpoints', '.pyHistory', 'AssignRandomPoints2KMLs.ipynb', 'backup.aprx', 'ImportLog', 'Index', 'LandCoverClassification_SamplingPlots.gdb', 'LandCoverClassification_SamplingPlots.lyrx', 'LandLandcov_ImpervSrfcs2016', 'Map1.pdf', 'Map1_CDL19.pdf', 'Map1_Imagery.pdf', 'Map_image.pdf', 'Map_wetland.pdf', 'New Notebook.ipynb', 'outputs', 'sites', 'temp', 'VSWI_Wetlands_Advisory_Layer', 'VSWI_Wetlands_Advisory_Layer.zip', 'VSWI_Wetlands_Class_Layer', 'VSWI_Wetlands_Class_Layer.zip', 'VTWetlands.aprx', 'VTWetlands.aprx.xml', 'VTWetlands.gdb', 'VTWetlands.tbx', 'VTWetlands_2021-03-04.aprx', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017', 'Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.zip']
C:/Workspace/VTWetlands/VTWetlands
C:/Workspace/VTWetlands/VTWetlands/outputs/
0      sralger
1     bbalpard
2     nebingha
3     sbradle2
4     lbrown28
        ...   
72      extra1
73      extra2
74      extra3
75      extra4
76      extra5

In [35]:
# define functions
def random_selection1(inlayer,nstudents,IDmin,z,npoints,col,netid,i):
    y = math.ceil(npoints*z)
    IDmax= npoints+ IDmin
    files = [None]*nstudents
    for x in range(nstudents): #for loop
        if x>0:
            IDmin=y+IDmin
            IDmax=IDmin+npoints
        print("index", x)
        print("student", netid[x])
        print(IDmin, IDmax)
        # set sql string
        sql=col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
        
        if x==nstudents : #for the last student wrap to the first student
            IDmax= IDmin + y
            sqllow=col+" > " +"0"+"  And "+col+" <= "+str(y)
            sqlhigh=" OR "+col+" > " +str(IDmin)+"  And "+col+" <= "+str(IDmax)
            sql=sqllow + sqlhigh
        print(sql)
        # select by attribute 
        
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points"+str(i)+"_random_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmin)

def sample_selection2(inlayer,nstudents,IDmin,z,npoints,col,netid,i,arr,nstudentsperpoint):
    IDmax= IDmin + npoints
    files = [None]*nstudents
    print(files)
    for x in range(nstudents): #for loop
        w = math.floor(x/nstudentsperpoint)
        if x == 0: wprior = w
        if w > wprior:
            wprior = w
            IDmin = IDmin+1
        IDmax = IDmin+npoints
            
        rands = arr[IDmin:IDmax]
        sql = " OR ".join(["%s = %s" % (col,str(r)) for r in rands])
        print("index", x)
        print("student", netid[x])
        print(w,rands)
        print(sql)
        # select by attribute 
        selected = arcpy.management.SelectLayerByAttribute(in_layer_or_view=inlayer,
                                        selection_type="SUBSET_SELECTION", 
                                        where_clause=sql)
        fname = "points"+str(i)+"_sample_"+netid[x]
        files[x] = arcpy.management.CopyFeatures(selected, "temp/"+fname)
        #arcpy.conversion.LayerToKML(layer=selected,
        #                            out_kmz_file=outfolder + "kmls/"+fname+".kmz")
        arcpy.management.Delete(selected)
    return(IDmax)

def globsearchinpath(inpath,globsearch):
    os.chdir(inpath)
    return(glob.glob(globsearch))

def os_cmd_ogr2ogr (inpath, outpath, outformat):
    cmd = "ogr2ogr -f \"{outformat}\" {outpath} {inpath}".format(outformat=outformat,
                                                                 outpath=outpath,
                                                                 inpath=inpath)
    print(cmd)
    try:
        os.system(cmd)
        print("success")
    except:
        print("error")
    return()

# extract values from list of rasters at points
def arcpy_globExtractMultiValues2Points(inpath,globsearch,inpoints):
    os.chdir(inpath)
    inrasters = glob.glob(globsearch)
    print(inrasters)
    return(arcpy.sa.ExtractMultiValuesToPoints(inpoints,inrasters))

## Prepare Floodplain Polygon Layer

In [36]:
# clip RCPP sites to 
infile = "Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017/Wetland_Restoration_Model_Site_Prioritization_Lake_Champlain_2017.shp"
clipfile = r"C:\Workspace\VTWetlands\VTWetlands\VTWetlands.gdb\Floodplain_500yr_drago_Proje"
clipped = outfolder+"RCCP_Floodplain_Clip.shp"
arcpy.analysis.Clip(infile, clipfile, clipped)

# dissolve polygons of constraining feature
inpath = clipped
print(inpath)
outfeatures = "Dissolved.shp"
outpath = outfolder + outfeatures
# check if file exists and delete before running script

if os.path.exists(outpath):
    arcpy.management.Delete(outpath)
if os.path.exists(inpath):
    arcpy.Dissolve_management (inpath, outpath)
else: 
    print("input file does not exist")


C:/Workspace/VTWetlands/VTWetlands/outputs/RCCP_Floodplain_Clip.shp


## Generate Random Points and Create Unique IDs

In [37]:
# create n random points inside one dissovled polygon
outname = "dissolved_randompoints.shp"
npoints = nstudents * 7 * 10
if os.path.exists(outfolder+outname):
    print("deleting ",outfolder+outname)
    arcpy.management.Delete(outfolder+outname)
random = arcpy.management.CreateRandomPoints(out_path=outfolder, 
                                    out_name=outname, 
                                    constraining_feature_class=outpath,             
                                    number_of_points_or_field=npoints)

random = arcpy.management.CopyFeatures(in_features=random,
                                       out_feature_class=wd+"VTWetlands.gdb/dissolved_random")

# create new field name for random points

codeblock = '''
def concat(x):
    return('r'+str(x))
    '''
expression = "concat(!OBJECTID!)"
inTable = random
fieldName = "Name"
random = arcpy.management.CalculateField(inTable, fieldName, expression, "PYTHON3", 
                                codeblock)
fieldList = arcpy.ListFields(random)  #get a list of fields for each feature class
print([fn.name  for fn in fieldList])

# create new field name for selected points

codeblock = '''
def concat(x):
    return('v'+str(x))
    '''
expression = "concat(!IDN!)"
selected = wd+"sites/mygeodata_csv_shp/selected_sites_2021-03-04-point.shp"
#newfeature = "copy_selected_sites"
fieldName = "Name"
selected = arcpy.management.CopyFeatures(in_features=selected,
                                         out_feature_class=wd+"VTWetlands.gdb/copy_selected_sites")
fieldList = arcpy.ListFields(selected)  #get a list of fields for each feature class
print([fn.name  for fn in fieldList])
"""for field in fieldList: #loop through each field
    if field.name.lower() == 'name':  #look for the name elev
        selected = arcpy.AlterField_management(selected, field.name, 'siteID')"""
selected = arcpy.management.CalculateField(selected, fieldName, expression, "PYTHON3", 
                                codeblock)

deleting  C:/Workspace/VTWetlands/VTWetlands/outputs/dissolved_randompoints.shp
['OBJECTID', 'Shape', 'CID', 'Name']
['OBJECTID', 'Shape', 'IDN', 'ID', 'lat', 'lon', 'sample_dat', 'project', 'Name']


## Extract  Data at Points 

In [38]:
names = ["copy_selected_sites","dissolved_random"]
files = [wd+"VTWetlands.gdb/"+"copy_selected_sites",
         wd+"VTWetlands.gdb/"+"dissolved_random"]
newname="merged_extracted"
print(files)

emptyFC = arcpy.CreateFeatureclass_management(wd+"VTWetlands.gdb",
                                              newname,
                                              "POINT", 
                                              files[0])

merge = arcpy.management.Merge(files,emptyFC, "", "ADD_SOURCE_INFO")

table=arcpy_globExtractMultiValues2Points(inpath="D:/GeoData/Cropscape_VT",
                                    globsearch="*.tif",
                                    inpoints=merge)
table=arcpy_globExtractMultiValues2Points(inpath='D:\\GeoData\\LandLandcov_BaseLC2016\\',
                                    globsearch="*.tif",
                                    inpoints=merge)
table=arcpy_globExtractMultiValues2Points(inpath=outfolder,
                                    globsearch="*.tif",
                                    inpoints=merge)

arcpy.management.CopyFeatures(in_features=table,
                              out_feature_class=outfolder+newname)
    
outpath=r"C:\\Workspace\\VTWetlands\\"
outname = r"merged_extracted.csv"
if os.path.exists(os.path.join(outpath,outname)):
    print(os.path.join(outpath,outname))
    os.remove(os.path.join(outpath,outname))
arcpy.conversion.TableToTable(table,outpath,outname)
    

['C:/Workspace/VTWetlands/VTWetlands/VTWetlands.gdb/copy_selected_sites', 'C:/Workspace/VTWetlands/VTWetlands/VTWetlands.gdb/dissolved_random']
['CDL_2008_clip_20201002155541_287304339.tif', 'CDL_2009_clip_20201002155541_287304339.tif', 'CDL_2010_clip_20201002155541_287304339.tif', 'CDL_2011_clip_20201002155541_287304339.tif', 'CDL_2012_clip_20201002155541_287304339.tif', 'CDL_2013_clip_20201002155541_287304339.tif', 'CDL_2014_clip_20201002155541_287304339.tif', 'CDL_2015_clip_20201002155541_287304339.tif', 'CDL_2016_clip_20201002155541_287304339.tif', 'CDL_2017_clip_20201002155541_287304339.tif', 'CDL_2018_clip_20201002155541_287304339.tif', 'CDL_2019_clip_20201002155541_287304339.tif']
['LandLandcov_BaseLC2016.tif']
['lclu16_agr.tif', 'lclu16_imperv.tif', 'lclu16_shr.tif', 'lclu16_wet.tif', 'rcpp_score.tif', 'vswi_advis.tif', 'vswi_class.tif', 'vtss_hygr.tif']
C:\\Workspace\\VTWetlands\\merged_extracted.csv


<Result 'C:\\Workspace\\VTWetlands\\merged_extracted.csv'>

## Randomly asign points to each student

In [39]:
tempdir = wd + "temp/"
#print(tempdir)
tempfiles = os.listdir(tempdir)
#print(tempfiles)
if os.path.exists(tempdir):
    try:
        os.remove(tempdir)
        os.mkdir(tempdir)
    except:
        "unable to delete tempdir"
        for file in tempfiles:
            filepath = os.path.join(tempdir,file)
            try:
                os.remove(filepath)
            except:
                print("unable to delete",filepath)


fdir = wd + "outputs/kmls/"
for file in tempfiles:
    filepath = os.path.join(fdir,file)
    try:
        os.remove(filepath)
    except:
        print("unable to delete",filepath)


# set starting parameters for loop
IDmin_sample = 0
IDmin_rand = 0
start = 0
stop = len(df.IDN)
arr = np.arange(start,stop)
print(arr)
netid_shuff = np.array(netid[:nstudents])
np.random.shuffle(netid_shuff)
print(netid_shuff)

unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.sbn
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.sbx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.shp
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.shp.xml
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/bbalpard.shx
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points0_random_bbalpard.cpg
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points0_random_bbalpard.dbf
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points0_random_bbalpard.prj
unable to delete C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/points0_random_bbalpard.sbn


In [40]:
for i in range(nreps):
    # shuffle netids to randomize plot assignments
    np.random.shuffle(netid_shuff)
    
    # RANDOM
    print ("iteration:",i)
    print ("IDmin_rand:",IDmin_rand)
    print ("IDmin_sample:",IDmin_sample)
    inlayer = random
    nstudents= len(roster[:nstudents])
    z = .75 # the proportion of nonoverlap, 
    npoints= 4
    col = "OBJECTID"
    
    IDmin_rand = random_selection1(inlayer,nstudents,IDmin_rand,z,npoints,col,netid_shuff,i)
       
    # SELECTED
    inlayer = selected
    np.random.shuffle(arr)
    # parameters
    nstudents= len(roster[:nstudents])
    z = 0 # the proportion of nonoverlap
    npoints= 1
    nstudentsperpoint = 9
    col = "OBJECTID"
    IDmin_sample = sample_selection2(inlayer,nstudents,IDmin_sample,z,
                                     npoints,col,netid_shuff,i,arr,nstudentsperpoint)



fpath = wd + "temp/"
for nid in netid[:nstudents]:
    print(nid)
    files = globsearchinpath(fpath,"points*"+nid+"*shp")
    
    # Process:  Create a new empty feature class to append shapefiles to
    print(files)
    
    
    emptyFC = arcpy.CreateFeatureclass_management(wd+"temp/",
                                              nid,
                                              "POINT", 
                                              files[0])
    merge = arcpy.management.Merge(files,emptyFC, "", "ADD_SOURCE_INFO")

    #copy = arcpy.management.CopyFeatures(merge, wd+"VTWetlands.gdb/"+outname)
    #arcpy.conversion.LayerToKML(layer=merge,
    #                            out_kmz_file=outfolder+"kmls/"+nid+".kmz")
    shppath=wd+"temp/"+nid+".shp"
    kmlpath=outfolder+"kmls/"+nid+".kml"

    os_cmd_ogr2ogr(outpath=kmlpath,inpath=shppath,outformat="KML")
    

iteration: 0
IDmin_rand: 0
IDmin_sample: 0
index 0
student eroy4
0 4
OBJECTID > 0  And OBJECTID <= 4
index 1
student dkager
3 7
OBJECTID > 3  And OBJECTID <= 7
index 2
student jsenacke
6 10
OBJECTID > 6  And OBJECTID <= 10
index 3
student ciobrien
9 13
OBJECTID > 9  And OBJECTID <= 13
index 4
student mdisaia
12 16
OBJECTID > 12  And OBJECTID <= 16
index 5
student gmccarth
15 19
OBJECTID > 15  And OBJECTID <= 19
index 6
student ntryan
18 22
OBJECTID > 18  And OBJECTID <= 22
index 7
student kchealy
21 25
OBJECTID > 21  And OBJECTID <= 25
index 8
student gjmartin
24 28
OBJECTID > 24  And OBJECTID <= 28
index 9
student extra5
27 31
OBJECTID > 27  And OBJECTID <= 31
index 10
student spurdy
30 34
OBJECTID > 30  And OBJECTID <= 34
index 11
student dullatho
33 37
OBJECTID > 33  And OBJECTID <= 37
index 12
student extra3
36 40
OBJECTID > 36  And OBJECTID <= 40
index 13
student msanange
39 43
OBJECTID > 39  And OBJECTID <= 43
index 14
student nebingha
42 46
OBJECTID > 42  And OBJECTID <= 46
inde

OBJECTID = 62
index 55
student awiegman
6 [62]
OBJECTID = 62
index 56
student bgroppe
6 [62]
OBJECTID = 62
index 57
student mkovalci
6 [62]
OBJECTID = 62
index 58
student pheaton
6 [62]
OBJECTID = 62
index 59
student amorton2
6 [62]
OBJECTID = 62
index 60
student cwalther
6 [62]
OBJECTID = 62
index 61
student ostern
6 [62]
OBJECTID = 62
index 62
student jkehmna
6 [62]
OBJECTID = 62
index 63
student mfriedb1
7 [49]
OBJECTID = 49
index 64
student extra2
7 [49]
OBJECTID = 49
index 65
student wkowalch
7 [49]
OBJECTID = 49
index 66
student jgess
7 [49]
OBJECTID = 49
index 67
student acady
7 [49]
OBJECTID = 49
index 68
student elparks
7 [49]
OBJECTID = 49
index 69
student cdemayo
7 [49]
OBJECTID = 49
index 70
student jkuhl
7 [49]
OBJECTID = 49
index 71
student nsteinth
7 [49]
OBJECTID = 49
index 72
student jlmashri
8 [15]
OBJECTID = 15
index 73
student kdorn
8 [15]
OBJECTID = 15
index 74
student mczimmer
8 [15]
OBJECTID = 15
index 75
student dncampbe
8 [15]
OBJECTID = 15
index 76
student epe

student emwong
3 [26]
OBJECTID = 26
index 30
student extra5
3 [26]
OBJECTID = 26
index 31
student dkager
3 [26]
OBJECTID = 26
index 32
student jspindel
3 [26]
OBJECTID = 26
index 33
student eschnei1
3 [26]
OBJECTID = 26
index 34
student epepe
3 [26]
OBJECTID = 26
index 35
student sralger
3 [26]
OBJECTID = 26
index 36
student jrickwoo
4 [41]
OBJECTID = 41
index 37
student ciobrien
4 [41]
OBJECTID = 41
index 38
student jdely
4 [41]
OBJECTID = 41
index 39
student extra4
4 [41]
OBJECTID = 41
index 40
student jlane4
4 [41]
OBJECTID = 41
index 41
student mgreally
4 [41]
OBJECTID = 41
index 42
student msetzke
4 [41]
OBJECTID = 41
index 43
student amhildeb
4 [41]
OBJECTID = 41
index 44
student kmillman
4 [41]
OBJECTID = 41
index 45
student acady
5 [17]
OBJECTID = 17
index 46
student jlmashri
5 [17]
OBJECTID = 17
index 47
student jkehmna
5 [17]
OBJECTID = 17
index 48
student rwatt
5 [17]
OBJECTID = 17
index 49
student elparks
5 [17]
OBJECTID = 17
index 50
student awpierce
5 [17]
OBJECTID = 17
i

OBJECTID = 44
index 4
student mkovalci
0 [44]
OBJECTID = 44
index 5
student msetzke
0 [44]
OBJECTID = 44
index 6
student dullatho
0 [44]
OBJECTID = 44
index 7
student eroy4
0 [44]
OBJECTID = 44
index 8
student jlmashri
0 [44]
OBJECTID = 44
index 9
student jrickwoo
1 [9]
OBJECTID = 9
index 10
student hnovembe
1 [9]
OBJECTID = 9
index 11
student cdemayo
1 [9]
OBJECTID = 9
index 12
student epepe
1 [9]
OBJECTID = 9
index 13
student amorton2
1 [9]
OBJECTID = 9
index 14
student jkehmna
1 [9]
OBJECTID = 9
index 15
student jgess
1 [9]
OBJECTID = 9
index 16
student ciobrien
1 [9]
OBJECTID = 9
index 17
student extra3
1 [9]
OBJECTID = 9
index 18
student cwalther
2 [48]
OBJECTID = 48
index 19
student hlavia
2 [48]
OBJECTID = 48
index 20
student ostern
2 [48]
OBJECTID = 48
index 21
student sralger
2 [48]
OBJECTID = 48
index 22
student szigic
2 [48]
OBJECTID = 48
index 23
student rwatt
2 [48]
OBJECTID = 48
index 24
student mdickhau
2 [48]
OBJECTID = 48
index 25
student spurdy
2 [48]
OBJECTID = 48
in

OBJECTID > 891  And OBJECTID <= 895
index 70
student sheny
894 898
OBJECTID > 894  And OBJECTID <= 898
index 71
student dullatho
897 901
OBJECTID > 897  And OBJECTID <= 901
index 72
student ciobrien
900 904
OBJECTID > 900  And OBJECTID <= 904
index 73
student extra2
903 907
OBJECTID > 903  And OBJECTID <= 907
index 74
student mswartl1
906 910
OBJECTID > 906  And OBJECTID <= 910
index 75
student mkovalci
909 913
OBJECTID > 909  And OBJECTID <= 913
index 76
student kchealy
912 916
OBJECTID > 912  And OBJECTID <= 916
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
index 0
student m

...

['points0_random_jdely.shp', 'points0_sample_jdely.shp', 'points1_random_jdely.shp', 'points1_sample_jdely.shp', 'points2_random_jdely.shp', 'points2_sample_jdely.shp', 'points3_random_jdely.shp', 'points3_sample_jdely.shp', 'points4_random_jdely.shp', 'points4_sample_jdely.shp', 'points5_random_jdely.shp', 'points5_sample_jdely.shp', 'points6_random_jdely.shp', 'points6_sample_jdely.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jdely.kml C:/Workspace/VTWetlands/VTWetlands/temp/jdely.shp
success
mfriedb1
['points0_random_mfriedb1.shp', 'points0_sample_mfriedb1.shp', 'points1_random_mfriedb1.shp', 'points1_sample_mfriedb1.shp', 'points2_random_mfriedb1.shp', 'points2_sample_mfriedb1.shp', 'points3_random_mfriedb1.shp', 'points3_sample_mfriedb1.shp', 'points4_random_mfriedb1.shp', 'points4_sample_mfriedb1.shp', 'points5_random_mfriedb1.shp', 'points5_sample_mfriedb1.shp', 'points6_random_mfriedb1.shp', 'points6_sample_mfriedb1.shp']
ogr2ogr -f "KML" C:/Workspace/

['points0_random_jkehmna.shp', 'points0_sample_jkehmna.shp', 'points1_random_jkehmna.shp', 'points1_sample_jkehmna.shp', 'points2_random_jkehmna.shp', 'points2_sample_jkehmna.shp', 'points3_random_jkehmna.shp', 'points3_sample_jkehmna.shp', 'points4_random_jkehmna.shp', 'points4_sample_jkehmna.shp', 'points5_random_jkehmna.shp', 'points5_sample_jkehmna.shp', 'points6_random_jkehmna.shp', 'points6_sample_jkehmna.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/jkehmna.kml C:/Workspace/VTWetlands/VTWetlands/temp/jkehmna.shp
success
kkoberna
['points0_random_kkoberna.shp', 'points0_sample_kkoberna.shp', 'points1_random_kkoberna.shp', 'points1_sample_kkoberna.shp', 'points2_random_kkoberna.shp', 'points2_sample_kkoberna.shp', 'points3_random_kkoberna.shp', 'points3_sample_kkoberna.shp', 'points4_random_kkoberna.shp', 'points4_sample_kkoberna.shp', 'points5_random_kkoberna.shp', 'points5_sample_kkoberna.shp', 'points6_random_kkoberna.shp', 'points6_sample_kkoberna.shp'

['points0_random_amorton2.shp', 'points0_sample_amorton2.shp', 'points1_random_amorton2.shp', 'points1_sample_amorton2.shp', 'points2_random_amorton2.shp', 'points2_sample_amorton2.shp', 'points3_random_amorton2.shp', 'points3_sample_amorton2.shp', 'points4_random_amorton2.shp', 'points4_sample_amorton2.shp', 'points5_random_amorton2.shp', 'points5_sample_amorton2.shp', 'points6_random_amorton2.shp', 'points6_sample_amorton2.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/amorton2.kml C:/Workspace/VTWetlands/VTWetlands/temp/amorton2.shp
success
hnovembe
['points0_random_hnovembe.shp', 'points0_sample_hnovembe.shp', 'points1_random_hnovembe.shp', 'points1_sample_hnovembe.shp', 'points2_random_hnovembe.shp', 'points2_sample_hnovembe.shp', 'points3_random_hnovembe.shp', 'points3_sample_hnovembe.shp', 'points4_random_hnovembe.shp', 'points4_sample_hnovembe.shp', 'points5_random_hnovembe.shp', 'points5_sample_hnovembe.shp', 'points6_random_hnovembe.shp', 'points6_samp

['points0_random_msanange.shp', 'points0_sample_msanange.shp', 'points1_random_msanange.shp', 'points1_sample_msanange.shp', 'points2_random_msanange.shp', 'points2_sample_msanange.shp', 'points3_random_msanange.shp', 'points3_sample_msanange.shp', 'points4_random_msanange.shp', 'points4_sample_msanange.shp', 'points5_random_msanange.shp', 'points5_sample_msanange.shp', 'points6_random_msanange.shp', 'points6_sample_msanange.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/msanange.kml C:/Workspace/VTWetlands/VTWetlands/temp/msanange.shp
success
eschnei1
['points0_random_eschnei1.shp', 'points0_sample_eschnei1.shp', 'points1_random_eschnei1.shp', 'points1_sample_eschnei1.shp', 'points2_random_eschnei1.shp', 'points2_sample_eschnei1.shp', 'points3_random_eschnei1.shp', 'points3_sample_eschnei1.shp', 'points4_random_eschnei1.shp', 'points4_sample_eschnei1.shp', 'points5_random_eschnei1.shp', 'points5_sample_eschnei1.shp', 'points6_random_eschnei1.shp', 'points6_samp

['points0_random_eroy4.shp', 'points0_sample_eroy4.shp', 'points1_random_eroy4.shp', 'points1_sample_eroy4.shp', 'points2_random_eroy4.shp', 'points2_sample_eroy4.shp', 'points3_random_eroy4.shp', 'points3_sample_eroy4.shp', 'points4_random_eroy4.shp', 'points4_sample_eroy4.shp', 'points5_random_eroy4.shp', 'points5_sample_eroy4.shp', 'points6_random_eroy4.shp', 'points6_sample_eroy4.shp']
ogr2ogr -f "KML" C:/Workspace/VTWetlands/VTWetlands/outputs/kmls/eroy4.kml C:/Workspace/VTWetlands/VTWetlands/temp/eroy4.shp
success
awiegman
['points0_random_awiegman.shp', 'points0_sample_awiegman.shp', 'points1_random_awiegman.shp', 'points1_sample_awiegman.shp', 'points2_random_awiegman.shp', 'points2_sample_awiegman.shp', 'points3_random_awiegman.shp', 'points3_sample_awiegman.shp', 'points4_random_awiegman.shp', 'points4_sample_awiegman.shp', 'points5_random_awiegman.shp', 'points5_sample_awiegman.shp', 'points6_random_awiegman.shp', 'points6_sample_awiegman.shp']
ogr2ogr -f "KML" C:/Workspace/

In [41]:
try:
    os.remove(tempdir)
    os.mkdir(tempdir)
except:
    print("unable to delete tempdir")

unable to delete tempdir
